In [11]:
from __future__ import division
import string
import os
import numpy as np
import caffe
import surgery
import os
import sys
import caffe
from datetime import datetime
from PIL import Image
import six.moves.cPickle as pickle
from operator import itemgetter, attrgetter
import scipy.io as sio


target = "trainval"
PATH_TO_OUTPUTS = "../siftflow-ensemble/"

if target == "trainval":
   PATH_TO_OUTPUTS = "../siftflow-distilled/"
else:
   PATH_TO_OUTPUTS = "../siftflow-ensemble/"

    
TST_TAG_LIST = ["overall accuracy", "mean accuracy", "mean IU", "fwavacc", "per-class IU", 'loss']
PATH_TO_SEM_LABELS = '../data/sift-flow/SemanticLabels/spatial_envelope_256x256_static_8outdoorcategories/'
PATH_TO_GEO_LABELS = '../data/sift-flow/GeoLabels/spatial_envelope_256x256_static_8outdoorcategories/'


In [9]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

def compute_hist(dataset, layer, n_cl):
    hist = np.zeros((n_cl, n_cl))
    for idx,name in enumerate(dataset):
        score = sio.loadmat(PATH_TO_OUTPUTS+'/'+ layer + '/' + name+'.mat')['S']
        if layer == 'score_sem':
            label = sio.loadmat(PATH_TO_SEM_LABELS + name +'.mat')['S']
        else:
            label = sio.loadmat(PATH_TO_GEO_LABELS + name +'.mat')['S']
        
        label = label.astype(np.uint8)
        label -= 1  # rotate labels so classes start at 0, void is 255    
        hist += fast_hist(label.flatten(), score.argmax(0).flatten(), n_cl)
    return hist

def do_seg_tests(dataset, layer, n_cl):
    ret = {x:-1 for x in TST_TAG_LIST}
    iter = 1
    hist = compute_hist(dataset, layer, n_cl)
    # overall accuracy
    acc = np.diag(hist).sum() / hist.sum()
    print '>>>', datetime.now(), 'Iteration', iter, 'overall accuracy', acc
    ret['overall accuracy'] = acc
    # per-class accuracy
    acc = np.diag(hist) / hist.sum(1)
    print '>>>', datetime.now(), 'Iteration', iter, 'mean accuracy', np.nanmean(acc)
    ret['mean accuracy'] = np.nanmean(acc)
    #per-class IU
    iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))
    print '>>>', datetime.now(), 'Iteration', iter, 'mean IU', np.nanmean(iu)
    ret['mean IU'] = np.nanmean(iu)
    #IU per class
    print 'Iteration', iter, 'per-class IU', iu
    ret['per-class IU'] = iu
    freq = hist.sum(1) / hist.sum()
    print '>>>', datetime.now(), 'Iteration', iter, 'fwavacc', (freq[freq > 0] * iu[freq > 0]).sum()
    ret['fwavacc'] = (freq[freq > 0] * iu[freq > 0]).sum()
    return ret
    

In [10]:
try:
    import setproctitle
    setproctitle.setproctitle(os.path.basename(os.getcwd()))
except:
    pass

# scoring
test = np.loadtxt('../data/sift-flow/'+target+'.txt', dtype=str)
print datetime.now(), "Running ..."
do_seg_tests(test, 'score_sem', 33)
do_seg_tests(test, 'score_geo', 3)


2017-02-09 20:30:19.310466 Running ...
>>> 2017-02-09 20:31:01.621571 Iteration 1 overall accuracy 0.868923941469
>>> 2017-02-09 20:31:01.636999 Iteration 1 mean accuracy 0.49814840777
>>> 2017-02-09 20:31:01.638483 Iteration 1 mean IU 0.391359234622
Iteration 1 per-class IU [  1.39330051e-01   0.00000000e+00   0.00000000e+00   2.43398393e-01
   2.01675546e-01   8.47635144e-01   0.00000000e+00   7.03474794e-01
   0.00000000e+00   3.39795036e-01   0.00000000e+00   2.99138265e-01
   3.38288900e-01   4.84292123e-01   3.92496150e-01              nan
   7.92358954e-01   2.17093293e-01   1.68990071e-01   0.00000000e+00
   6.25658164e-01   8.52908289e-01   2.30503744e-01   5.67965833e-01
   7.82390566e-01   5.19678121e-01   3.06972962e-01   9.33875339e-01
   6.81818182e-01   6.74081564e-04   8.32501782e-01   7.09878418e-01
   3.10703307e-01]
>>> 2017-02-09 20:31:01.641319 Iteration 1 fwavacc 0.782576187253


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:28: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: RuntimeWarning: invalid value encountered in true_divide


>>> 2017-02-09 20:31:09.934090 Iteration 1 overall accuracy 0.9510228962
>>> 2017-02-09 20:31:09.934841 Iteration 1 mean accuracy 0.950982079204
>>> 2017-02-09 20:31:09.935764 Iteration 1 mean IU 0.90264804309
Iteration 1 per-class IU [ 0.93406579  0.86313651  0.91074183]
>>> 2017-02-09 20:31:09.937364 Iteration 1 fwavacc 0.907004203949


{'fwavacc': 0.90700420394934511,
 'loss': -1,
 'mean IU': 0.90264804308979729,
 'mean accuracy': 0.95098207920402011,
 'overall accuracy': 0.95102289620002889,
 'per-class IU': array([ 0.93406579,  0.86313651,  0.91074183])}